## Set intersecton performance

In [1]:
import numpy as np

def gen_random_set(set_len, choice_len=10000):
    return set(np.random.choice(choice_len, set_len, replace=False))
    

def gen_sets(set_count: set, set_len=100, choice_len=10000) -> list:
    return [gen_random_set(set_len, choice_len) 
            for _ in range(set_count)]


In [8]:
gen_sets(5, set_len=10)

[{227, 836, 2534, 3080, 4245, 5120, 6351, 7501, 7907, 8185},
 {2465, 2706, 3831, 4123, 4573, 4924, 5462, 5702, 5851, 6507},
 {1716, 1781, 1888, 2540, 4078, 4205, 4572, 5035, 7354, 9465},
 {56, 956, 2042, 2699, 3597, 3841, 4990, 6283, 6897, 7677},
 {1639, 2118, 4285, 4311, 4886, 5012, 5177, 6857, 7297, 8880}]

In [2]:
sets = gen_sets(100000, set_len=100, choice_len=500)
test_set = gen_random_set(100, choice_len=500)

In [26]:
def process_sets(sets: list):
    intersections = []
    for s in sets:
        intersection = s & test_set
        if len(intersection) > 20:
            intersections.append(intersection)    

In [43]:
%timeit process_sets(sets)

1 loop, best of 3: 349 ms per loop


### With numpy matrices

In [1]:
import numpy as np

cols = 100
rows = 100000
arr = np.random.choice(cols * 5, cols * rows).reshape(rows, cols)
tst = np.random.choice(cols * 5, cols)
arr

array([[229, 346, 309, ..., 440, 412, 321],
       [ 67, 105, 489, ..., 194, 325, 122],
       [347, 367, 236, ..., 198,  56, 307],
       ..., 
       [489, 375, 298, ..., 153, 461,  61],
       [410, 390, 281, ..., 191, 437, 241],
       [262, 209,  10, ..., 247, 460,  69]])

In [ ]:
%%timeit

tf = np.isin(arr, tst)
intersections = np.count_nonzero(tf, axis=1)
active_sets = []
for i, ins_count in enumerate(intersections):
    if ins_count > 20:
        active_sets.append(set(arr[i]))

In [57]:
len(active_sets)

278

In [13]:
import numpy as np

cols = 10
rows = 100000
arr = np.random.choice(2, cols * rows).reshape(rows, cols)
tst = np.random.choice(2, cols)
print(arr)
print(tst)

[[0 0 0 ..., 0 0 1]
 [0 1 0 ..., 1 0 0]
 [0 0 1 ..., 0 1 1]
 ..., 
 [1 1 0 ..., 1 0 0]
 [0 0 0 ..., 0 0 0]
 [0 1 0 ..., 0 1 1]]
[1 1 1 1 1 1 0 1 1 1]


In [14]:

intersections = arr & tst
ones = np.count_nonzero(intersections, axis=1)


In [21]:
print(ones)
print(np.where(ones > 8))

[2 2 5 ..., 5 1 5]
(array([   73,   181,   668,  1029,  2709,  3580,  3766,  4411,  4469,
        4632,  4847,  5192,  5422,  5825,  5863,  5987,  6048,  6126,
        6836,  7529,  7986,  9203,  9572,  9623, 10416, 10536, 11305,
       11563, 12116, 12327, 12831, 14686, 15189, 15729, 17088, 17731,
       19260, 19371, 19405, 19458, 20953, 22476, 22726, 23036, 24893,
       25593, 26877, 27468, 29912, 30326, 30852, 31146, 31227, 31471,
       32401, 33589, 34343, 34911, 35270, 35285, 35839, 36852, 37076,
       37749, 38411, 38632, 38848, 39141, 39463, 39561, 39889, 40151,
       41267, 42777, 43541, 43779, 44779, 44812, 44972, 46707, 46776,
       47284, 47381, 49407, 50125, 50630, 50741, 50815, 51365, 51581,
       51794, 52122, 52190, 52626, 52942, 53969, 54703, 55506, 55886,
       55916, 56311, 56329, 56739, 56777, 58954, 59050, 59085, 59785,
       59863, 60557, 60595, 61031, 61085, 61143, 62096, 62830, 62917,
       63092, 65230, 65549, 65861, 66261, 67142, 67375, 68233, 68844,


In [26]:
arr =arr = np.empty((0,3), int)
arr = np.vstack((arr, np.array([1,2,3])))
arr = np.vstack((arr, np.array([4,5,6])))
arr

array([[1, 2, 3],
       [4, 5, 6]])

## BitArray
https://pypi.python.org/pypi/bitarray/

In [14]:

from bitarray import bitarray

ba = bitarray(8)
ba2 = bitarray(8)
# ba.setall(0)
# ba[0] = True
print(ba.__sizeof__())
print(ba, ba2)
ba.count(1)

72
bitarray('11101000') bitarray('11101000')


4

In [12]:
from bitarray import bitarray
import numpy as np

def gen_random_bit_array(bit_count=100, active_bit_count=50):
    bit_indices = np.random.choice(bit_count, active_bit_count, replace=False)
    bits = np.zeros(bit_count, dtype=np.bool)
    bits[bit_indices] = True
    return bitarray(bits.tolist())

In [13]:
%%time
bit_rows = [gen_random_bit_array() for _ in range(100000)]
tst_bits = gen_random_bit_array()
print(tst_bits)

bitarray('1011110011000010010100100101100010111000001110011100000011011000111111110111011110011010001000001101')
Wall time: 3.39 s


In [48]:
%%timeit
intersections = []
for idx, bits in enumerate(bit_rows):
    intersection = bits & tst_bits
    if intersection.count(1) > 20:
        intersections.append(intersection)

10 loops, best of 3: 50.9 ms per loop


In [63]:
intersections

[bitarray('0110100000000000001010010000000010000000011000100000100000010100001101000000110110010100100000000110'),
 bitarray('1010100100000010100100010010000000000001011000000000000010000101001001000100100010000100010000000000'),
 bitarray('0010000100000110100000010010000101010001011000000000000010010101001001100100110000010000000000000010'),
 bitarray('1010000100010000100000010000000000010000110000000000000010000101001000100100011011100000000000000110'),
 bitarray('0110100000000110000010000000000011000001010000000000100010010001011000000000011000010000111000010010'),
 bitarray('0010100100010100000100000000000111010001001000000000100010000001001100100100110110010100000000000000'),
 bitarray('0110000000000110001010000000001100010001000000100000000000000000010000100000110011110100001000000010'),
 bitarray('0110100100010110100010010010001000010000000000100000000010000000011100000000100100010000101000010100'),
 bitarray('100000010000010000000000000000101001000011000000000010000001000101110

In [61]:
bits_1 = gen_random_bit_array()
bits_2 = gen_random_bit_array()
intersection = bits_1 & bits_2
print(bits_1)
print(bits_2)
print(intersection)
print(intersection.count(1))

bitarray('0101100110011110111011100111101010100101010001111000000100010011111000100001101010100101001000110111')
bitarray('1001111101100111101010000000111010100010000001010001111110010111001001010011111011001000100110111010')
bitarray('0001100100000110101010000000101010100000000001010000000100010011001000000001101010000000000000110010')
26


## Enums performance

In [3]:
from enum import Enum

COLOR_RED = 1
COLOR_GREEN = 2
COLOR_BLUE = 3

class Color(Enum):
    red = 1
    green = 2
    blue = 3

In [17]:
%%timeit
color1 = Color.red
color2 = Color.green
color3 = Color.blue

The slowest run took 25.82 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 351 ns per loop


In [16]:
%%timeit
color1 = COLOR_RED
color2 = COLOR_GREEN
color3 = COLOR_BLUE

The slowest run took 36.00 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 48.8 ns per loop


## Iterators

In [4]:
def generator(count: int=10):
    for i in range(count):
        yield i

In [5]:
list(generator())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [8]:
def letter_to_note(letter):
    return ord(letter.lower()) - ord('a')

# generator
def take_phrase_generator(text: str, phrase_len) -> list:
    for i in range(0, len(text) - phrase_len):
        yield [letter_to_note(letter) 
               for letter in text[i:i + phrase_len]]

In [9]:
text = 'ucheniteotkrilicherazlikatasedlzhinamikrobitekoitosekriiatvnaskhorataspodeliatsamoedniisshchimikrobiavsichkiostanalisastrogoindividualni'
take_phrase = take_phrase_generator(text, 5)
for i in range(10):
    print(next(take_phrase))

[20, 2, 7, 4, 13]
[2, 7, 4, 13, 8]
[7, 4, 13, 8, 19]
[4, 13, 8, 19, 4]
[13, 8, 19, 4, 14]
[8, 19, 4, 14, 19]
[19, 4, 14, 19, 10]
[4, 14, 19, 10, 17]
[14, 19, 10, 17, 8]
[19, 10, 17, 8, 11]


## Vector sum

In [161]:
def bit_average_sum(vecs: np.array) -> np.array:
    return np.sum(vecs, axis=0) / vecs.shape[0]
    
def after_norm_sum(vecs: np.array) -> np.array:
    vec_sum = np.sum(vecs, axis=0)
    norm = np.max(vec_sum)  # np.linalg.norm(vec_sum)
    return vec_sum / norm

def inter_norm_sum(vecs: np.array) -> np.array:
    vec_sum = np.ones(cols)
    for vec in vecs:
        vec_sum += vec
        norm = np.max(vec_sum)  # np.linalg.norm(vec_sum)
        vec_sum /= norm
    return vec_sum

In [178]:
rows = 15
cols = 5
vecs = np.random.choice(2, cols * rows).reshape(rows, cols)
print(vecs)

print(bit_average_sum(vecs))
print(after_norm_sum(vecs))
print(inter_norm_sum(vecs))

np.random.shuffle(vecs)
print(vecs)
print(inter_norm_sum(vecs))

[[1 1 0 1 0]
 [1 0 0 0 1]
 [1 1 0 1 1]
 [0 0 1 0 1]
 [1 1 0 0 1]
 [0 1 0 1 1]
 [1 1 1 1 1]
 [1 1 0 1 0]
 [1 0 1 1 0]
 [0 1 0 0 0]
 [1 1 1 1 0]
 [0 0 1 0 1]
 [1 0 0 1 0]
 [0 0 0 0 1]
 [0 0 1 0 1]]
[ 0.6         0.53333333  0.4         0.53333333  0.6       ]
[ 1.          0.88888889  0.66666667  0.88888889  1.        ]
[ 0.3491388   0.13738602  0.73647416  0.35045593  1.        ]
[[1 1 0 1 1]
 [1 0 0 1 0]
 [0 0 1 0 1]
 [1 1 1 1 1]
 [1 1 1 1 0]
 [1 0 0 0 1]
 [0 0 1 0 1]
 [1 1 0 1 0]
 [0 1 0 0 0]
 [1 1 0 0 1]
 [1 1 0 1 0]
 [0 0 1 0 1]
 [0 0 0 0 1]
 [0 1 0 1 1]
 [1 0 1 1 0]]
[ 0.72217939  0.42534053  0.74376767  1.          0.61937805]


## Multiporcessing

In [ ]:
from multiprocessing import Pool
def f(x):
    return x**2

pool = Pool(4)

for res in pool.map(f,range(20)):
    print(res)

## Numpy array as key

In [2]:

import numpy as np

vector = np.random.choice(2, 1000)

In [104]:
%%time
for _ in range(10000):
    tuple_key = tuple(vector)

Wall time: 374 ms


In [46]:
tuple_key.__sizeof__()

3912

In [134]:
%%time
for _ in range(10000):
    str_key = "".join(['1' if bit > 0 else '0' for bit in vector])

Wall time: 1.87 s


In [135]:
%%time
for _ in range(10000):
    str_key = vector.tobytes()

Wall time: 9 ms


In [50]:
str_key.__sizeof__()

4033

In [122]:
str_key

b'\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [133]:
"".join(['0' if c == '\x00' else '1' for c in str_key.decode()])

'100010000000100010001000100010001000100000001000100000000000100010001000100010000000100000000000000010000000100010000000100000000000000000001000100010000000000000001000000010000000000010001000100010001000000010000000100000000000100010000000000000000000100010001000000010000000000000000000100000000000100000000000100010000000100010001000000010000000100010000000000000000000000010000000100010000000000000001000100000000000000010001000000000001000000010001000100010001000000000000000100010000000100010000000100010000000100010000000000000000000000010001000000010001000100000001000000000001000000010001000000000000000000000001000000010001000000000000000100010001000000000001000100000000000100010000000100010000000100000001000000010000000100000000000000010000000000000000000100000000000000010000000100010000000000010001000000000001000100010000000100000000000100010000000100010000000100010001000000010000000000010000000100000000000000010001000000000000000000000000000100010001000100010001000000000000000100

## Bit sum

In [235]:
def popcount(x):
    x -= (x >> 1) & 0x5555555555555555
    x = (x & 0x3333333333333333) + ((x >> 2) & 0x3333333333333333)
    x = (x + (x >> 4)) & 0x0f0f0f0f0f0f0f0f
    return ((x * 0x0101010101010101) & 0xffffffffffffffff ) >> 56

In [280]:
import numba

@numba.njit
def bitcount(x):
    return (1 & x) + (1 & x >> 1) + (1 & x >> 2) + (1 & x >> 3) + (1 & x >> 4) + (1 & x >> 5) + (1 & x >> 6) + (1 & x >> 7)

In [311]:
#     int v = value - ((value >> 1) & 0x55);
#     v = (v & 0x33) + ((v >> 2) & 0x33);
#     return ((v + (v >> 4) & 0x0F));
#     // 0x5 = 0101 (bit) 0x55 = 01010101
#     // 0x3 = 0011 (bit) 0x33 = 00110011
#     // 0xF = 1111 (bit) 0x0F = 00001111

@numba.njit
def fast_bitcount(x):
    x -= (x >> 1) & 0x55
    x = (x & 0x33) + ((x >> 2) & 0x33)
    return (x + (x >> 4)) & 0x0F

v_bitcount = np.vectorize(fast_bitcount)

In [276]:
x = 11
print(bin(x >> 1 & 1))
print(bitcount(x))
(11 & 1) + ((11 >> 1) & 1)

0b1
0


2

In [490]:
import numpy as np

rows = 100000
cols = 256
N = cols * rows
arr = np.random.choice(2, N)
arr = arr.reshape(rows, cols)
print(arr)
print(np.sum(arr, axis=1))
pkarr = np.packbits(arr, axis=1)
print(pkarr.size)
pkarr

[[0 1 0 ..., 1 0 1]
 [0 0 1 ..., 0 0 0]
 [1 1 0 ..., 1 1 0]
 ..., 
 [0 0 1 ..., 1 1 1]
 [1 1 0 ..., 1 0 0]
 [0 0 1 ..., 1 1 0]]
[119 117 126 ..., 111 126 119]
3200000


array([[ 82,  74, 237, ..., 117, 129,  93],
       [ 41, 110,  52, ..., 220, 179,  96],
       [222, 110, 121, ...,  88,  71, 134],
       ..., 
       [ 37,  66,  82, ...,  34, 219,  87],
       [216,  91, 110, ..., 162, 127, 196],
       [ 56,  86,  94, ...,  35,  39, 102]], dtype=uint8)

In [504]:
%%time
np.sum(np.apply_along_axis(v_bitcount, axis=1, arr=pkarr), axis=1)

Wall time: 2.14 s


array([119, 117, 126, ..., 111, 126, 119])

In [505]:
%%time
np.sum(arr, axis=1)

Wall time: 39 ms


array([119, 117, 126, ..., 111, 126, 119])

In [491]:
row = np.random.choice(2, cols)
pkrow = np.packbits(row)

In [496]:
%%time
arr & row

Wall time: 78 ms


array([[0, 1, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 1, 1, 0],
       ..., 
       [0, 0, 0, ..., 1, 1, 1],
       [1, 1, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 1, 0]], dtype=int32)

In [501]:
%%time
pkarr & pkrow

Wall time: 11 ms


array([[ 64,   8, 228, ...,  36, 129,  69],
       [  9,  44,  36, ...,  12, 131,  64],
       [200,  44,  96, ...,   8,   7, 134],
       ..., 
       [  1,   0,  64, ...,  32, 131,  71],
       [200,   9, 100, ...,  32,   7, 196],
       [  8,   4,  68, ...,  32,   7,  70]], dtype=uint8)

## Vector norm & nearest

In [19]:
import numpy as np
cols = 20
rows = 10
N = cols * rows
a = np.random.choice(2, N)
a = a.reshape(rows, cols)
a

array([[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
       [0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
       [1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1],
       [1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1],
       [1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1]])

In [20]:
norm = np.linalg.norm(a, axis=1)

an = a / norm[:, None]

np.linalg.norm(an, axis=1)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [36]:
v = np.random.choice(2, cols)
vn = v / np.linalg.norm(v)
print(v)
vn

[1 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 1 1 0]


array([ 0.35355339,  0.        ,  0.        ,  0.        ,  0.35355339,
        0.35355339,  0.        ,  0.35355339,  0.        ,  0.35355339,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.35355339,  0.35355339,  0.35355339,  0.        ])

In [39]:
sim = np.dot(an, vn.T)
print(sim)
print('vec:', v)
print('max:', a[np.argmax(sim)])
print('v & max:', v & a[np.argmax(sim)])
print('v | max:', v | a[np.argmax(sim)])
print('min:', a[np.argmin(sim)])
print('v & min:', v & a[np.argmin(sim)])

[ 0.42640143  0.35355339  0.375       0.47434165  0.31980107  0.31980107
  0.61237244  0.2236068   0.11785113  0.37796447]
vec: [1 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 1 1 0]
max: [1 0 1 0 0 1 1 1 0 1 0 1 1 1 0 0 0 1 1 1]
v & max: [1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1 1 0]
v | max: [1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 1 1 1]
min: [0 1 1 0 0 0 0 0 1 0 1 0 1 1 0 1 0 0 1 1]
v & min: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
